In [ ]:
#make sure you have these versions of TensorFlow and Keras

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import keras
keras.__version__

'2.15.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
train=pd.read_csv('/content/drive/Shareddrives/ presentation/train.csv')
val=pd.read_csv('/content/drive/Shareddrives/ presentation/val.csv')
test=pd.read_csv('/content/drive/Shareddrives/ presentation/test.csv')





In [ ]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  999 non-null    object
 1   content    999 non-null    object
dtypes: object(2)
memory usage: 15.7+ KB


In [ ]:
data = pd.concat([train ,  val , test])
data.columns = [ "sentiment","content"]

In [ ]:
unique_sentiments = data['sentiment'].unique()
print(unique_sentiments)

['neutral' 'worry' 'surprise' 'hate' 'empty' 'happiness' 'sadness' 'fun'
 'enthusiasm' 'love' 'relief' 'boredom' 'anger']


In [ ]:
sentiment_mapping = {
    'empty': 'negative',
    'sadness': 'negative',
    'worry': 'negative',
    'hate': 'negative',
    'boredom': 'negative',
    'anger': 'negative',
    'enthusiasm': 'positive',
    'neutral': 'neutral',
    'surprise': 'neutral',
    'love': 'positive',
    'fun': 'positive',
    'happiness': 'positive',
    'relief': 'positive'
}

# Create a new col for the mapping
data['label'] = data['sentiment'].map(sentiment_mapping)

In [ ]:
# Drop the original column, 'sentiment'
data = data.drop(columns=['sentiment'])

In [ ]:
unique_sentiments = data['label'].unique()
print(unique_sentiments)

['neutral' 'negative' 'positive']


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 1998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  40000 non-null  object
 1   label    40000 non-null  object
dtypes: object(2)
memory usage: 937.5+ KB


In [ ]:
data.rename(columns={'content': 'text'}, inplace=True)
#data.rename(columns={'sentiment': 'label'}, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 1998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  object
dtypes: object(2)
memory usage: 937.5+ KB


In [ ]:
data.shape

(40000, 2)

In [ ]:
data.isna().any(axis=1).sum()

0

In [ ]:
#text preprocessing
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #leave only characters from a to z
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #trun list into sentences
    return " ".join(review)


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data['text']=data['text'].apply(lambda x: preprocess(x))


In [ ]:
data['label']
#data['text']

0        neutral
1        neutral
2       negative
3       negative
4        neutral
          ...   
1994    negative
1995    negative
1996     neutral
1997     neutral
1998    negative
Name: label, Length: 40000, dtype: object

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data['N_label'] = label_encoder.fit_transform(data['label'])


In [ ]:
data['text']

0       came hous morn stink cat poo cat busi littl bi...
1       bah bk recept comp aint work sun got hat hip h...
2       bbc malaria parasit becom resist drug http tru...
3       morn appear bit sore head perhap bag pork scra...
4                book appoint give blood pm nd june scare
                              ...                        
1994          think hire one tranlsat one ever understand
1995                     http twitpic com w see mile away
1996                              desper go venic beinnal
1997                                  sleep work tomorrow
1998               ouch epic bruis toe lump ankl bad time
Name: text, Length: 40000, dtype: object

In [ ]:
# Creating the Bag of Words model by applying Countvectorizer -convert textual data to numerical data


from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))#example: the course was long-> [the,the course,the course was,course, course was, course was long,...]

data_cv = cv.fit_transform(data['text']).toarray()




In [ ]:
data_cv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test=data_cv,test_cv,train['N_label'],test['N_label']
X_train, X_test, y_train, y_test =train_test_split(data_cv, data['N_label'], test_size=0.25, random_state=42)


In [ ]:
# Create classifiers
logreg = LogisticRegression(max_iter=1200)

random_forest = RandomForestClassifier()
linear_svc = SVC(kernel='linear')
# Create a StackingClassifier
estimators = [('rf', random_forest), ('svc', linear_svc)]
stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=logreg)
# Train and evaluate each classifier using cross-validation
classifiers = [logreg, random_forest, linear_svc, stacking_classifier]
for classifier in classifiers:
    # Cross-validation scores
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=5)
    avg_cv_score = sum(cv_scores) / len(cv_scores)
    # Train the classifier
    classifier.fit(X_train, y_train)
    # Predict on the test set
    y_pred = classifier.predict(X_test)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Classifier: {classifier.__class__.__name__}")
    print(f"Cross-Validation Average Score: {avg_cv_score:.3f}")
    print(f"Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-Score: {f1:.3f}")
    print("-" * 50)

Classifier: LogisticRegression
Cross-Validation Average Score: 0.551
Accuracy: 0.552, Precision: 0.548, Recall: 0.552, F1-Score: 0.550
--------------------------------------------------
Classifier: RandomForestClassifier
Cross-Validation Average Score: 0.545
Accuracy: 0.544, Precision: 0.540, Recall: 0.544, F1-Score: 0.542
--------------------------------------------------
